In [1]:
import pandas as pd
import numpy as np

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [17]:
# Ventanas de tres dias
Xs = {}
print("Cargando features")
for ventana_nro in range(1, 7):
    try:
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_relaciones.csv".format(ventana_nro), index_col=0)
        print("Ventana {} agregada".format(ventana_nro))
    except FileNotFoundError:
        print("No se econtro la ventana {}, se generará".format(ventana_nro))
        Xs[ventana_nro] = installs.loc[installs["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
Ventana 1 agregada
Ventana 2 agregada
Ventana 3 agregada
Ventana 4 agregada
Ventana 5 agregada
Ventana 6 agregada


## Set que se usará para predecir

In [19]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_relaciones.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir


In [20]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[6].columns.values)

---

# Carga de datos

In [2]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

In [3]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

In [4]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

In [5]:
ventanas = []
for x in range(1,8):
    ventanas.append("ventana_{}".format(x))

In [6]:
columns = ["date","device_id","dia"] + ventanas
auctions = auctions[columns]
installs = installs[columns]

In [7]:
#Para distinguir despues del merge 
auctions.rename(columns ={"date": "date_auction"},inplace = True)
installs.rename(columns ={"date": "date_install"},inplace = True)

In [8]:
install_auctions = pd.merge_asof(auctions,installs, left_on = "date_auction", 
                               right_on = "date_install", by = "device_id",
                               suffixes = ("_auction","_install"))

In [9]:
del auctions

In [10]:
install_auctions = install_auctions[~install_auctions["date_install"].isnull()]

In [11]:
install_auctions = install_auctions[~install_auctions["date_auction"].isnull()]

---

### De auctions- events me quedo con sólo datos que correspondan a la competencia

In [14]:
install_auctions_competencia = install_auctions.loc[install_auctions["device_id"].isin(targets_competencia.index)]

----

### Función generadora de features

In [24]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final,nombre_data_merge):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        print("generando feature en v{}".format(ventana_nro))
       
        condicion = dataframe["ventana_{}_install".format(ventana_nro)] & dataframe["ventana_{}".format(ventana_nro) +"_"+nombre_data_merge]
        
        feature = generador_feature(dataframe.loc[condicion], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")
        destinos[ventana_nro].fillna(0,inplace = True)

---

## Auctions - Events

## Cantidad de eventos luego de la ultima subastas dentro de la ventana

In [21]:
def cantidad_subastas_desde_ultima_instalacion(dataframe,nro_ventana): 
    idx = dataframe.groupby(["device_id"])["date_install"].transform(max) == dataframe["date_install"]
    dataframe = dataframe[idx]
    
    feature = dataframe.groupby(["device_id"])["date_auction"].agg({"date_auction":"count"})\
                    .rename(columns = {"date_auction":"cantidad_subastas_desde_ultima_instalacion_hasta_fin_ventana"})
     
    return feature["cantidad_subastas_desde_ultima_instalacion_hasta_fin_ventana"]


In [23]:
features["cantidad_subastas_desde_ultima_instalacion_hasta_fin_ventana"] = (cantidad_subastas_desde_ultima_instalacion, ["cantidad_subastas_desde_ultima_instalacion_hasta_fin_ventana"])

### Genero los features

In [25]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(install_auctions, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7,"auction")
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(install_auctions_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8, "auction")
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature cantidad_subastas_desde_ultima_instalacion_hasta_fin_ventana en ventanas
generando feature en v1


/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


generando feature en v2
generando feature en v3
generando feature en v4
generando feature en v5
generando feature en v6
Generando feature cantidad_subastas_desde_ultima_instalacion_hasta_fin_ventana en set de predicciones
generando feature en v7


---

In [29]:
del install_auctions
del install_auctions_competencia

## Installs - Events

In [30]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [31]:
ventanas = []
for x in range(1,8):
    ventanas.append("ventana_{}".format(x))

In [32]:
columns = ["date","device_id","dia"] + ventanas
events = events[columns]

In [33]:
#Para distinguir despues del merge 
events.rename(columns ={"date": "date_event"},inplace = True)

In [34]:
install_events = pd.merge_asof(events,installs, left_on = "date_event", 
                               right_on = "date_install", by = "device_id",
                               suffixes = ("_event","_install"))

In [35]:
del events

In [36]:
install_events = install_events[~install_events["date_install"].isnull()]

In [37]:
install_events = install_events[~install_events["date_event"].isnull()]

In [42]:
install_events_competencia = install_events.loc[install_events["device_id"].isin(targets_competencia.index)]

---

In [46]:
def cantidad_eventos_desde_ultima_instalacion(dataframe,nro_ventana): 
    idx = dataframe.groupby(["device_id"])["date_install"].transform(max) == dataframe["date_install"]
    dataframe = dataframe[idx]
    
    feature = dataframe.groupby(["device_id"])["date_event"].agg({"date_event":"count"})\
                    .rename(columns = {"date_event":"cantidad_eventos_desde_ultima_instalacion_hasta_fin_ventana"})
     
    return feature["cantidad_eventos_desde_ultima_instalacion_hasta_fin_ventana"]


In [47]:
features["cantidad_eventos_desde_ultima_instalacion_hasta_fin_ventana"] = (cantidad_eventos_desde_ultima_instalacion, ["cantidad_eventos_desde_ultima_instalacion_hasta_fin_ventana"])

### Genero los features

In [48]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(install_events, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7,"event")
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(install_events_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8, "event")
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature cantidad_eventos_desde_ultima_instalacion_hasta_fin_ventana en ventanas
generando feature en v1


/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


generando feature en v2
generando feature en v3
generando feature en v4
generando feature en v5
generando feature en v6
Generando feature cantidad_eventos_desde_ultima_instalacion_hasta_fin_ventana en set de predicciones
generando feature en v7


In [51]:
del install_events
del install_events_competencia

---

# Install- clicks

In [52]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

In [53]:
ventanas = []
for x in range(1,8):
    ventanas.append("ventana_{}".format(x))

In [54]:
columns = ["date","device_id","dia"] + ventanas
clicks = clicks[columns]

In [55]:
#Para distinguir despues del merge 
clicks.rename(columns ={"date": "date_click"},inplace = True)

In [56]:
install_clicks = pd.merge_asof(clicks,installs, left_on = "date_click", 
                               right_on = "date_install", by = "device_id",
                               suffixes = ("_click","_install"))

In [58]:
del clicks

In [59]:
install_clicks = install_clicks[~install_clicks["date_install"].isnull()]

In [60]:
install_clicks = install_clicks[~install_clicks["date_click"].isnull()]

In [61]:
install_clicks_competencia = install_clicks.loc[install_clicks["device_id"].isin(targets_competencia.index)]

---

In [63]:
def cantidad_clicks_desde_ultima_instalacion(dataframe,nro_ventana): 
    idx = dataframe.groupby(["device_id"])["date_install"].transform(max) == dataframe["date_install"]
    dataframe = dataframe[idx]
    
    feature = dataframe.groupby(["device_id"])["date_click"].agg({"date_click":"count"})\
                    .rename(columns = {"date_click":"cantidad_clicks_desde_ultima_instalacion_hasta_fin_ventana"})
     
    return feature["cantidad_clicks_desde_ultima_instalacion_hasta_fin_ventana"]


In [64]:
features["cantidad_clicks_desde_ultima_instalacion_hasta_fin_ventana"] = (cantidad_clicks_desde_ultima_instalacion, ["cantidad_clicks_desde_ultima_instalacion_hasta_fin_ventana"])

### Genero los features

In [65]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(install_clicks, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7,"click")
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(install_clicks_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8, "click")
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature cantidad_clicks_desde_ultima_instalacion_hasta_fin_ventana en ventanas
generando feature en v1
generando feature en v2
generando feature en v3
generando feature en v4
generando feature en v5
generando feature en v6
Generando feature cantidad_clicks_desde_ultima_instalacion_hasta_fin_ventana en set de predicciones
generando feature en v7


/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


In [67]:
for ventana_nro in range(1, 7):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_relaciones.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_relaciones.csv")